<a href="https://colab.research.google.com/github/vsadykov/kineticsim_reader/blob/main/ML_VDFmoments_modeltesting_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing modules and dependences

In [13]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Preparing the data set

In [14]:
featurevector_allvdfs_all = np.load('allsimulations.mldata_vdfs.npy')
labels_allvdfs_all = np.load('allsimulations.mldata_labels.npy')
timep_array_all = np.load('allsimulations.mldata_timep.npy')

Log normalization of the VDFs

In [15]:
featurevector_allvdfs_all = np.log10(featurevector_allvdfs_all + 1)
for t in range (0, featurevector_allvdfs_all.shape[0], 1):
  featurevector_allvdfs_all[t,0,:,:] /= np.amax(featurevector_allvdfs_all[t,0,:,:])
  featurevector_allvdfs_all[t,1,:,:] /= np.amax(featurevector_allvdfs_all[t,1,:,:])

In [16]:
X_train, X_test, f_train, f_test = train_test_split(featurevector_allvdfs_all, labels_allvdfs_all, \
                                                    test_size=0.33, random_state=1)

In [17]:
print("Shapes of the data sets:")
print(X_train.shape, f_train.shape)
print(X_test.shape, f_test.shape)

Shapes of the data sets:
(667, 2, 40, 40) (667,)
(330, 2, 40, 40) (330,)


# Development and training of the CNN prototype

In [18]:
class VDFCNN(nn.Module):
  def __init__(self):
    super(VDFCNN, self).__init__()
    self.cnncell = nn.Sequential(
        nn.Conv2d(1, 2, kernel_size=3, padding=1),
        nn.ReLU(True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(2, 4, kernel_size=3, padding=1),
        nn.ReLU(True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(4, 4, kernel_size=3, padding=1),
        nn.ReLU(True),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )
    self.linearcell = nn.Sequential(
        nn.Linear(4*5*5, 10),
        nn.ReLU(True),
        nn.Linear(10,2),
        nn.ReLU(True)
    )
  def forward(self, x):
    x = self.cnncell(x)
    x = x.view(-1, 4 * 5 * 5)
    x = self.linearcell(x)
    return x